In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install Unidecode

     |████████████████████████████████| 245kB 5.2MB/s 


In [ ]:
train_df_path   = "/content/drive/MyDrive/CASE STUDY - 1/submission/Factorization Machine/train.tsv"
test2_df_path   = "/content/drive/MyDrive/CASE STUDY - 1/submission/Factorization Machine/test_stg2.tsv"

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import regex as re
import unidecode
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from nltk.stem import PorterStemmer 
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, Normalizer, LabelEncoder
from sklearn.model_selection import train_test_split
import scipy
from scipy.sparse import csr_matrix
from nltk import word_tokenize, pos_tag, ne_chunk
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow import keras
import itertools
import multiprocessing
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
import gc
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Input, GRU, LSTM, Dense, BatchNormalization, Dropout, RNN, Flatten, GlobalAveragePooling1D, concatenate, PReLU, Concatenate
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1, l2, l1_l2
import pickle


num_cores = multiprocessing.cpu_count()
num_cores


2

In [ ]:
train_df  = pd.read_table(train_df_path)
test_df = pd.read_table(test2_df_path)

In [ ]:
train_df['brand_name'].fillna("unknown", inplace=True)
test_df['brand_name'].fillna("unknown", inplace=True)

train_df['category_name'].fillna("unknown/unknown/unknown", inplace=True)
test_df['category_name'].fillna("unknown/unknown/unknown", inplace=True)

train_df['item_description'].fillna("no description yet", inplace=True)
test_df['item_description'].fillna("no description yet", inplace=True)

train_df['cat_1'] = train_df.category_name.str.split('/').str.get(0).astype('category')
train_df['cat_2'] = train_df.category_name.str.split('/').str.get(1).astype('category')
train_df['cat_3'] = train_df.category_name.str.split('/').str.get(2).astype('category')

test_df['cat_1'] = test_df.category_name.str.split('/').str.get(0).astype('category')
test_df['cat_2'] = test_df.category_name.str.split('/').str.get(1).astype('category')
test_df['cat_3'] = test_df.category_name.str.split('/').str.get(2).astype('category')


In [ ]:
global setofBrands
setofBrands = np.unique(train_df['brand_name'].astype(str).str.lower())


global setofCat1
setofCat1 = np.unique(train_df['cat_1'].astype(str).str.lower())

global setofCat2
setofCat2 = np.unique(train_df['cat_2'].astype(str).str.lower())

global setofCat3
setofCat3 = np.unique(train_df['cat_3'].astype(str).str.lower())

In [ ]:
# https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe

# concat cat_1, cat_2, cat_3 
def concat_c1_c2_c3(c_1, c_2, c_3):
    return c_1 + "/" + c_2 + "/" + c_3


# filling missing cat_1 column from item_description and name
def miss_cat1(row):
    
    cat1 = row['cat_1']
    
    if row['cat_1']=='unknown':
        
        for i in setofCat1:
            if i in row['item_description'].lower():
                
                # print("1 desc {}".format(i))
                
                cat1 = i
                break
            elif i in row['name'].lower():
                
                # print("1 name {}".format(i))
                
                cat1 = i
                break   
    
    return cat1


## filling missing cat_2 column from item_description and name
def miss_cat2(row):
    
    cat2 = row['cat_2']
    
    if row['cat_2']=='unknown':
        
        for i in setofCat2:
            if i in row['item_description'].lower():
                
                # print("2 desc {}".format(i))
                
                cat2 = i
                break
            elif i in row['name'].lower():
                
                # print("2 name {}".format(i))
                
                cat2 = i
                break   
    
    return cat2


## filling missing cat_3 column from item_description and name
def miss_cat3(row):
    
    cat3 = row['cat_3']
    
    if row['cat_3']=='unknown':
        
        for i in setofCat3:
            if i in row['item_description'].lower():
                
                # print("3 desc {}".format(i))
                
                cat3 = i
                break
            elif i in row['name'].lower():
                
                # print("3 name {}".format(i))
                
                cat3 = i
                break   
    
    return cat3
    

## get missing brand name from name and description
def miss_brand(row):
    
    brand_name = row['brand_name']
    desc = row['item_description'].lower()
    name = row['name'].lower()
    
    if row['brand_name']=='unknown':
        
        for i in setofBrands:
            if i in desc:
                                
                brand_name = i
                break
            elif i in name:
                
                brand_name = i
                break   
    
    return brand_name


def fill_missing_data(df):
    
    df['cat_1']         = df.apply(miss_cat1, axis=1)
    df['cat_2']         = df.apply(miss_cat2, axis=1)
    df['cat_3']         = df.apply(miss_cat3, axis=1)
    df['brand_name']    = df.apply(miss_brand, axis=1)
    df['category_name'] = df.apply(lambda x: concat_c1_c2_c3(str(x.cat_1), str(x.cat_2), str(x.cat_3)), axis=1)
    
    return df

In [ ]:

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_cores)
    with Pool(num_cores) as pool:
        df = pd.concat(pool.map(func, df_split))
    return df


In [ ]:
%%time
train_df = parallelize_dataframe(train_df, fill_missing_data)
train_df.isnull().any()

CPU times: user 4.76 s, sys: 1.52 s, total: 6.28 s
Wall time: 3min 51s


In [ ]:
%%time
test_df = parallelize_dataframe(test_df, fill_missing_data)
test_df.isnull().any()

CPU times: user 11.2 s, sys: 4.05 s, total: 15.3 s
Wall time: 8min 58s


In [ ]:
num_word_dict = {
    
    "0":"zero",
    "1":"one",
    "2":"tow",
    "3":"three",
    "4":"four",
    "5":"five",
    "6":"six",
    "7":"seven",
    "8":"eight",
    "9":"nine",
    '+':'plus'
}

def number_to_word(text):
    sentence = ''
    for letter in text:
        try:
            
            if letter.isdigit():
                sentence = sentence + ' ' + num_word_dict[letter]
            else:
                sentence = sentence + letter
        except:
            sentence = sentence + ''
            
    return sentence

def accented_to_english(text):
    return unidecode.unidecode(text)

def fill_description(text):
    if len(text) == 0:
        return "no description yet"
    else:
        return text

def fill_name(text):
    if len(text) == 0:
        return "no name"
    else:
        return text
    
def remove_stop_words(sent):
    stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]
    
    sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
    return sent

def decontracted(phrase):
    
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    return phrase

def remove_line(sent):
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    
    return sent

def setmming(sent):
    ps = PorterStemmer()

def preprocess_cleaning(df):
    
    df['name'] = df['name'].apply(decontracted)
    df['name'] = df['name'].apply(remove_line)
    df['name'] = df['name'].apply(accented_to_english)
    df['name'] = df['name'].str.replace(r'[^A-Za-z0-9 ]', r' ')
    # df['name'] = df['name'].apply(number_to_word)
    df['name'] = df['name'].str.strip()
    df['name'] = df['name'].str.replace(' +', ' ')
    df['name'] = df['name'].str.lower()
    df['name'] = df['name'].apply(fill_name)
    
  
    df['brand_name'] = df['brand_name'].str.replace(r'[^A-Za-z0-9 ]', r' ')# remove all char except words and letters
    df['brand_name'] = df['brand_name'].str.strip()# removing leading and tailing spaces
    df['brand_name'] = df['brand_name'].str.replace(' +', ' ')# replacing double_space with single space
    df['brand_name'] = df['brand_name'].str.replace(' ', '_')# replacing space with under_score
    df['brand_name'] = df['brand_name'].str.lower()# converting to lowercase
    

    df['cat_1'] = df['cat_1'].apply(accented_to_english)
    df['cat_1'] = df['cat_1'].str.replace(r'[^A-Za-z0-9 ]', r' ')# remove all char except words and letters
    df['cat_1'] = df['cat_1'].apply(remove_line)
    df['cat_1'] = df['cat_1'].str.strip()# removing leading and tailing spaces
    df['cat_1'] = df['cat_1'].str.replace(' +', ' ')# replacing double_space with single space
    df['cat_1'] = df['cat_1'].str.replace(' ', '_')# replacing space with under_score
    df['cat_1'] = df['cat_1'].str.lower()# converting to lowercase
 
    df['cat_2'] = df['cat_2'].apply(accented_to_english)
    df['cat_2'] = df['cat_2'].str.replace(r'[^A-Za-z0-9 ]', r' ')# remove all char except words and letters
    df['cat_2'] = df['cat_2'].str.strip()# removing leading and tailing spaces
    df['cat_2'] = df['cat_2'].str.replace(' +', ' ')# replacing double_space with single space
    df['cat_2'] = df['cat_2'].str.replace(' ', '_')# replacing space with under_score
    df['cat_2'] = df['cat_2'].str.lower()# converting to lowercase
    

    df['cat_3'] = df['cat_3'].apply(accented_to_english)
    df['cat_3'] = df['cat_3'].str.replace(r'[^A-Za-z0-9 ]', r' ')# remove all char except words and letters
    df['cat_3'] = df['cat_3'].str.strip()# removing leading and tailing spaces
    df['cat_3'] = df['cat_3'].str.replace(' +', ' ')# replacing double_space with single space
    df['cat_3'] = df['cat_3'].str.replace(' ', '_')# replacing space with under_score
    df['cat_3'] = df['cat_3'].str.lower()# converting to lowercase
    
    df['category_name'] = df['cat_1'] + "_" + df['cat_2'] + "_" + df['cat_3']
    
    df['item_description'] = df['item_description'].apply(decontracted)
    df['item_description'] = df['item_description'].apply(remove_line)
    df['item_description'] = df['item_description'].apply(accented_to_english)
    df['item_description'] = df['item_description'].str.replace(r'[^a-zA-Z0-9 ]', r' ')# remove all char except words and letters
    df['item_description'] = df['item_description'].str.strip()# removing leading and tailing spaces
    df['item_description'] = df['item_description'].str.replace(' +', ' ')# replacing double_space with single space
    df['item_description'] = df['item_description'].str.lower()# converting to lowercase
    df['item_description'] = df['item_description'].apply(fill_description)
    
    return df


In [ ]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_cores)
    with Pool(num_cores) as pool:
        df = pd.concat(pool.map(func, df_split))
    return df

In [ ]:
%%time
train_df = parallelize_dataframe(train_df, preprocess_cleaning)

CPU times: user 6.8 s, sys: 2.29 s, total: 9.08 s
Wall time: 3min


In [ ]:

train_test_save_path = "/content/drive/MyDrive/CASE STUDY - 1/data/"

train_df.to_csv(train_test_save_path+"train__cleaned.csv", index=False)


In [ ]:
del train_df

In [ ]:
gc.collect()

472

In [ ]:
%%time
test_df = parallelize_dataframe(test_df, preprocess_cleaning)


CPU times: user 15.5 s, sys: 7.03 s, total: 22.5 s
Wall time: 7min 8s


In [ ]:

train_test_save_path = "/content/drive/MyDrive/CASE STUDY - 1/data/"


In [ ]:

train_test_save_path = "/content/drive/MyDrive/CASE STUDY - 1/data/"

test_df.to_csv(train_test_save_path+"test__cleaned.csv", index=False)
